# Scrapper de sites

> Récupère le contenu de chacun des URL classé par typologie (variable ndf dans le code), chargé depuis le fichier Sites.json.

> Stocke le résultat dans ndf qui conditionne la variable "fichier de sortie". Celle-ci peut être adaptée pour pointer sur une zone correcte de votre drive dans la zone "Personnalisation" dénotée ci-dessous. Les données des sites collectés sont stockées au format pickle dans le dossier /OUT/ContenusSites classé par typologie selon Sites.json.

> La variable de sortie est de type dictionnaire pour conserver les url sources (flexibilité pour pouvoir corriger). Cf. commentaires infra. 




In [ ]:
# Il faut initialiser le dossier à partir de GitHub
# cette action est à faire une seule fois lors de la première utilisation
!git clone https://github.com/Patent2net/LexiComWebPartners.git

Cloning into 'LexiComWebPartners'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 29 (delta 8), reused 27 (delta 6), pack-reused 0
Unpacking objects: 100% (29/29), done.


In [ ]:
!git clone https://github.com/Patent2net/LexiComWeb.git

Cloning into 'LexiComWeb'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 76 (delta 11), reused 73 (delta 8), pack-reused 0
Unpacking objects: 100% (76/76), done.


In [ ]:
stockageEntree = "/content/LexiComWeb/"
stockageEntreePartners = "/content/LexiComWebPartners/"
stockageSortie = "/content/LexiComWebPartners/"

In [ ]:
import os, urllib
os.chdir('/content/LexiComWeb/RESSOURCES/')
from outils import isPartner, text_from_html, myRequest


---
# Personnalisation (Option)

> Ces cellules permettent de réaliser les traitements à partir de son propre espace de stockage. A n'exécuter que dans ce cas en adaptant les dossiers d'entrée et sortie. NE PAS EXECUTER SAUF A VOULOIR PERSONNALISER LES TRAITEMENTS.

Si vous avez exécuté le scraper et reconstitué l'ensemble des dossiers nécessaires sur votre drive, ces cellules vous permettent de configurer ce notebook pour travailler sur vos données et non celles issues du git (accessibles sur un dossier virtuel via le menu *Fichiers* à gauche).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Récupérer les entrées sur son drive
stockageEntree = "/content/drive/MyDrive/OUT"

In [ ]:
# Récupérer les sorties sur son drive (créer un dossier "OUT")
stockageSortie = "/content/drive/MyDrive/OUT"

---

In [ ]:
# recup import requests, re, pickle
from bs4 import BeautifulSoup
from urllib import parse 
import requests
import re
import json


> Le fichier niveau1-PNPC.csv a été réalisé avec Hyphe pour collecter depuis le site du PNPC (http://www.portcros-parcnational.fr/fr) l'ensemble des liens sortants. Le site pointé par chacun de ces liens est récursivement collecté jusqu'à un niveau de profondeur 3 pour déterminer l'existence d'un lien retour vers le PNPC. La liste des urls de ce fichier consigne ces adresses de sites considérés alors comme des partenaires de la communication Web.



In [ ]:
with open (stockageEntreePartners + "RESSOURCES/NIVEAU1-PNPC.csv", "r") as partners:
  donnee= partners.readlines()
partenaires = []
for lig in donnee[1:]:
  urls = lig.split(";")[2] # 3e colonne
  partenaires.extend(urls.split()) # liste d'URLs séparés par des espaces

> Hyphe récupère une liste de partenaires (urls) que le logiciel consigne selon plusieurs protocoles. Quelquefois le chemin est vide (racine du site) d'autres non... Pour pouvoir tester l'appartenance d'un site à cette liste nous normalisons ces urls et l'étendons pour satisfaire la variété du web. Nous retenons ici qu'un site web peut être identifié par les urls sous les formes suivantes (deux protocoles avec ou sans 'www' dans la dénomination) :

*   http(s)://site.extension
*   http(s)://www.site.extension

La variable partenairesPropres sert à les extrapoler à partir des données de Hyphe.



In [ ]:
partenairesPropres = []
for url in set(partenaires):
  url = url.strip()
  url = url.replace('"', "")

  urlP = parse.urlparse(url)
  if 'https' in urlP.scheme:
    if 'www.' in urlP.hostname:
      partenairesPropres.append(urlP.scheme + '://' + urlP.hostname)
      partenairesPropres.append(urlP.scheme + '://' + urlP.hostname.replace('www.', ''))
      partenairesPropres.append('http://' + urlP.hostname.replace('www.', ''))
      partenairesPropres.append('http://' + urlP.hostname)

    else:
      partenairesPropres.append(urlP.scheme + '://www.' + urlP.hostname)
      partenairesPropres.append(urlP.scheme + '://' + urlP.hostname)
      partenairesPropres.append('http://www.' + urlP.hostname)
      partenairesPropres.append('http://' + urlP.hostname)
  else:
    if 'www.' in urlP.hostname:
      partenairesPropres.append(urlP.scheme + '://' + urlP.hostname)
      partenairesPropres.append(urlP.scheme + '://' + urlP.hostname.replace('www.', ''))
      partenairesPropres.append('https://' + urlP.hostname.replace('www.', ''))
      partenairesPropres.append('https://' + urlP.hostname)

    else:
      partenairesPropres.append(urlP.scheme + '://www.' + urlP.hostname)
      partenairesPropres.append(urlP.scheme + '://' + urlP.hostname)
      partenairesPropres.append('https://www.' + urlP.hostname)
      partenairesPropres.append('https://' + urlP.hostname)

partenairesPropres = list(set(partenairesPropres)) # sans doublons

In [ ]:
print(len(set(partenairesPropres)))

356


> recup des déjà collectés

In [ ]:
fichierDeSortie = stockageEntree + '/ContenusPartners/' + 'Partners.json'
with open (fichierDeSortie, 'r', encoding ='utf8') as  fictemp: #
  listePartners = json.load(fictemp)

In [ ]:
import urllib3

urllib3.disable_warnings() # pourquoi les certificats SSL ne passent pas aujourd'hui ? 30/11 

# Première boucle pour lever les problèmes

> La dynamique du web fait que certains sites peuvent ne pas être accessibles, ne pas répondre à un instant t, ou faire planter le collecteur. Ce qui suit teste chaque URL et construit la variable BadUrl avec les urls en erreur.

La liste des url collectés et étendue par le procédé précédent génère aussi des adresses pas forcément valides. Ce qui suit est fait pour les expurger.

In [ ]:
listePartners = dict()

In [ ]:
BadUrl = []
Done = []
for url in partenaires: 
  if url not in listePartners .keys() and isPartner(url, partenairesPropres+partenaires):
      try:
        webpage = myRequest(url)
      except:
        print("bad", url)
        BadUrl.append(url)

bad http://www.fr-fr.facebook.com/pages
bad https://www.fr-fr.facebook.com/pages
bad https://alizee-soft.com
bad https://www.alizee-soft.com
bad https://port-cros-parcnational.fr
bad https://www.port-cros-parcnational.fr
bad http://www.espritparcnationalportcros.com
bad https://www.espritparcnationalportcros.com
bad http://var.gouv.fr
bad https://var.gouv.fr
bad https://www.var.gouv.fr
bad https://www.festival-galathea.com
bad https://bateaux-taxi.com
bad http://obi1-portcros-parcnational.fr
bad http://www.obi1-portcros-parcnational.fr
bad https://obi1-portcros-parcnational.fr
bad https://www.obi1-portcros-parcnational.fr
bad http://wixsite.com
bad https://wixsite.com
bad http://marches-publics.gouv.fr
bad http://www.marches-publics.gouv.fr
bad https://marches-publics.gouv.fr
bad http://univ-tln.fr
bad https://univ-tln.fr
bad https://cbnmed.fr
bad https://www.cbnmed.fr
bad http://superdoc.com
bad http://www.superdoc.com
bad https://superdoc.com
bad https://www.superdoc.com
bad http://w

In [ ]:
print (len(BadUrl ))

36


In [ ]:
done = [] 
listePartners = dict()

In [ ]:
for url in partenaires: 
  webpage ="" 
  if url not in listePartners .keys() and isPartner(url, partenairesPropres):
    if url not in BadUrl and url not in done:
      #soup = dict() # changement de type de données pour conserver l'URL source
  # récupère l'URL d'un site web et enregistre la page web
      del(webpage)
      try:
        webpage = myRequest(url)
      except:
        BadUrl .append(url)
        urlTemp = url
        url = url .replace('http:', "https:")
        print ("test https ", url)
        try:
          webpage = myRequest(url)
        except:
          BadUrl .append(url)
          if 'www' not in url:
            url = url .replace('https://', "https://www.")
          try:
            webpage = myRequest(url)
            print ("sans www ?")
          except:
              print ("rien à faire ", url)
              BadUrl .append(url)
              continue
  # récupère le contenu de la page web à l'aide de BeautifulSoup
    #soup.append(BeautifulSoup(webpage.content, "html.parser"))
      tempoSoup = BeautifulSoup(webpage.content, "html.parser")
      if tempoSoup.title is not None:
        titre = tempoSoup.title.text 
      else:
        titre = ""
      if tempoSoup.description is not None:
        desc = tempoSoup.description.text 
      else:
        desc = ""
      texte = text_from_html(webpage.content)
      if len(titre)>0 and len(desc)>0:
        listePartners[url] = titre + '\n' + desc + "\n" + texte
      elif len(titre)>0:
        listePartners[url] = titre + "\n" + texte
      elif len(desc)>0:
        listePartners[url] = desc + "\n" + texte
      else:
        listePartners[url] = texte
      # le contenu est dans le "casier" nommé par l'url
    # nettoyage
    
    # soup [cle] = re.sub('^a-zA-Z0-9àâäèéêëîïôœùûüÿçÀÂÄÈÉÊËÎÏÔŒÙÛÜŸÇ', ' ', soup [cle])
      re.sub('^a-zA-ZàâäèéêëîïôœùûüÿçÀÂÄÈÉÊËÎÏÔŒÙÛÜŸÇ', ' ', listePartners [url])
      listePartners[url] = listePartners[url].replace("\xa0", " ")
      listePartners[url] = listePartners[url].replace("\n", " ")
      listePartners[url] = listePartners[url].replace("’", "'")
      listePartners[url] = listePartners[url].translate('utf8')
      dictionary = {"\\": ""}
      transtable= listePartners[url].maketrans(dictionary)
      listePartners[url] = listePartners[url].translate(transtable)
      listePartners[url] = str(listePartners[url])
      
    # c'est pas tip top. Certains caractères restent. Variable selon les sites. 

    # sauvegarde
      fichierDeSortie = stockageSortie + '/ContenusPartners/Partners.json'
      with open (fichierDeSortie, 'w') as  fictemp: #on met tous les contenus dans un pickle
        json.dump(listePartners, fictemp)
    done .append (url)

test https  https://risque-prevention-incendie.fr
rien à faire  https://www.risque-prevention-incendie.fr
test https  https://www.risque-prevention-incendie.fr
rien à faire  https://www.risque-prevention-incendie.fr


In [ ]:
len(set(listePartners.values()))

102

## Commentaires et exemple de ce qui est récupéré


*  Les clés du dictionnaire de sortie sont les url
*  Le contenu du dictionnaire pour une clé donne le texte récupéré
*  Il peut y avoir des doublons de contenu (cas des sites qui répondent à deux URL équivalente (avec et sans www par ex.).
* Ce dernier point sera traité dans l'instrument suivant (TraiteContenuPartners.ipynb)




In [ ]:
listePartners [url]

"Accueil - Sauvegarde des forets varoises               A propos   Qui sommes-nous  Découvrir le projet COPAINS    Nos services   Espaces naturels  Espaces verts  Propreté urbaine  Nettoyage des plages    Nos produits bio   Tous nos produits  Où trouver nos produits ?  Le panier Porquerollais  Conditions de Livraison    Agenda  Devenir partenaire   Particuliers  Entreprises  Nos partenaires    Contact  0 Article           Sélectionner une page                             Sauvegarde Des Forêts Varoises Sauvegarde des Forêts Varoises,  acteur local de l'environnement au service de l'emploi et du développement des territoires depuis 1991.         Pour la préservation de l'environnement et l'inclusion dans l'emploi             Une Economie Sociale et Solidaire au service du  développement des territoires et de l'emploi   Nous sommes engagés sur des actions de préservation et de valorisation de notre environnement dans le cadre d'un projet solidaire qui soutient l'insertion par l'emploi.   